In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import DataFrameStatFunctions
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext
dbutils.library.installPyPI("missingpy", version="0.2.0")

PyPI package missingpy has been installed already. The previously-installed package is `missingpy==0.2.0`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.
Out[354]: False

In [2]:
spark = SparkSession.builder.appName('LendingClub').getOrCreate()

In [3]:
DataFrame = sqlContext.sql('select * from loan_csv')

In [4]:
display(DataFrame)

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,null,10+ years,RENT,24000.0,Verified,12/1/2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077501,Borrower added on 12/22/11 > I need to upgrade my business technologies.,credit_card,Computer,860xx,AZ,27.65,0,1/1/1985,1,null,null,3,0,13648,83.7,9,f,0.0,0.0,5861.071414,5831.78,5000.0,861.07,0.0,0.0,0.0,1/1/2015,171.62,null,1/1/2016,0,null,1,INDIVIDUAL,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,12/1/2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077430,Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike. I only need this money because the deal im looking at is to good to pass up. Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike.I only need this money because the deal im looking at is to good to pass up. I have finished college with an associates degree in business and its takingmeplaces,car,bike,309xx,GA,1.0,0,4/1/1999,5,null,null,3,0,1687,9.4,4,f,0.0,0.0,1008.71,1008.71,456.46,435.17,0.0,117.08,1.11,4/1/2013,119.66,null,9/1/2013,0,null,1,INDIVIDUAL,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,null,10+ years,RENT,12252.0,Not Verified,12/1/2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077175,null,small_business,real estate business,606xx,IL,8.72,0,11/1/2001,2,null,null,2,0,2956,98.5,10,f,0.0,0.0,3003.653644,3003.65,2400.0,603.65,0.0,0.0,0.0,6/1/2014,649.91,null,1/1/2016,0,null,1,INDIVIDUAL,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,12/1/2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1076863,"Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace. I'm very sorry to let my loan expired last time.",other,personel,917xx,CA,20.0,0,2/1/1996,1,35,null,10,0,5598,21.0,37,f,0.0,0.0,12226.30221,12226.3,10000.0,2209.33,16.97,0.0,0.0,1/1/2015,357.48,null,1/1/2015,0,null,1,INDIVIDUAL,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,12/1/2011,Current,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1075358,"Borrower added on 12/21/11 > I plan on combining three large interest bills togeth

#Feature selection, Feature Engineering and Preprocessing

In [6]:
# for c in DataFrame.columns:
display(DataFrame.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in DataFrame.columns)))


id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,0,0,0,0,0,0,0,0,51457,0,0,4,0,0,0,0,0,761350,0,151,0,0,0,29,29,29,454312,750326,29,29,0,502,29,0,0,0,0,0,0,0,0,0,0,17659,0,252971,53,145,665676,0,0,886868,886870,886868,29,70276,70276,866007,866007,866007,866007,866569,866007,868762,866007,866007,866007,866007,70276,866007,866007,866007


** Removing the columns with highest number(600000+) of null values as they don't contribute anything to the analysis, when compared to the total number of rows(890000)

In [8]:
df = DataFrame.drop('member_id','emp_title','url','desc','title','zip_code','annual_inc_joint','dti_joint','verification_status_joint','open_acc_6m','open_il_6m','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','inq_fi','total_cu_tl','inq_last_12m','mths_since_last_record','mths_since_last_major_derog')

Dropping active loans as we are building a supervised machine learning model with good and bad as labels

In [10]:
import pyspark.sql.functions as f

df_test = df.filter((f.col('loan_status') == "Current") | (f.col('loan_status') == 'Issued'))

In [11]:

df1 = df.filter((f.col('loan_status') != "Current"))

In [12]:
df_main = df1.filter((f.col('loan_status') != "Issued"))

In [13]:
#checking the count of holdout set for out of sample testing and current test set
print(df_test.count(),df_main.count())

610239 277140

In [14]:
df2 = df_main.sample(True,0.5)

#Visualizing the data for better understanding

In [16]:
df_ts = df2

In [17]:
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date
from pyspark.sql.functions import month
#dates to time stamp
df_ts = df_ts.withColumn("issue_d_ts",to_date(df_ts.issue_d,'MM/dd/yyyy')).\
                      withColumn('earliest_cr_line_ts',to_date(df_ts.earliest_cr_line, 'MM/dd/yyyy'))
                      



In [18]:
df_ts =  df_ts.withColumn("issue_year",year(df_ts.issue_d_ts)).\
                      withColumn('earliest_cr_line_year',year(df_ts.earliest_cr_line_ts)).\
                      withColumn("issue_month",month(df_ts.issue_d_ts))
                      


Categorizing good and bad loans

In [20]:
from pyspark.sql import functions as F
df_ts = df_ts.withColumn("Loan_status",F.when(col("loan_status") == "Charged Off",'Bad')
                    .when(col("loan_status") == "Default",'Bad')
                    .when(col("loan_status") == "In Grace Period",'Bad')
                    .when(col("loan_status") == "Does not meet the credit policy. Status:Fully Paid",'Bad')
                    .when(col("loan_status") == "In Grace Period",'Bad')
                    .when(col("loan_status") == "Late (31-120 days)",'Bad')
                    .otherwise('Good'))

In [21]:
df_ts = df_ts.withColumn("Emp_Length",regexp_extract(col('emp_length'),'\d+',0))
df_ts = df_ts.withColumn("emp_length",df_ts['Emp_Length'].cast("int"))

In [22]:
#Selecting good and bad loans using temp tables
df_ts.registerTempTable("temp_table")
df_ts = sqlContext.sql('select * from temp_table where loan_status="Good" or  loan_status = "Bad"')
df_ts.registerTempTable("loans_timestamp")

Our main motto behind categorising the loans to good and bad is to see what is the trend of number of loans in these categories across years and interpret that on future implications

##### Purpose of loan vs count of purpose

In [25]:
display(sqlContext.sql("select purpose, count(purpose)as count_p, avg(funded_amnt) as Average_Loan_Amount_Funded from loans_timestamp group by purpose  order by count_p desc limit 10"))

purpose,count_p,Average_Loan_Amount_Funded
debt_consolidation,81786,14508.786650526985
credit_card,26991,14063.205327701826
home_improvement,8049,13613.427133805442
other,7817,9352.760010234106
major_purchase,3322,9949.593618302228
small_business,2637,15090.851346226773
car,1906,8044.805876180482
medical,1599,9125.234521575985
moving,1114,7417.392280071814
wedding,1057,10443.472090823085


###### From the above plot we can see that the number of loans taken by the purpose of *Debt Consolidation* is highest with count of 81164, the total average amount of loans taken by *Small Businesses* is highest. So not only the number of loans taken matters but also the amount that has been aggregated due to the loan purpose is important

##### Issuance of Average Loan amount funded across each year

In [28]:
display(sqlContext.sql("select issue_year as Year, avg(funded_amnt) as Average_Loan_Amount_Funded, Loan_Status from loans_timestamp group by issue_year,Loan_Status order by issue_year asc"))

Year,Average_Loan_Amount_Funded,Loan_Status
2007,9085.06711409396,Good
2007,7401.206896551724,Bad
2008,8142.067307692308,Bad
2008,8473.902925531915,Good
2009,9626.759530791789,Good
2009,10166.28486055777,Bad
2010,10278.679653679654,Bad
2010,10072.774693721287,Good
2011,11382.459578637923,Good
2011,12896.819620253165,Bad


###### We can see from the above graph the average amount of loan taken has been gradually increased across each year which is expected as the income value and the expenditure values has been increase along the years. Also we can see that the average amount of good and bad loans almost has the same proportion across the years

##### Loan Status Good or Bad  across each Year

In [31]:
%sql
select issue_year as Year, loan_status, count(loan_status) as Loan from loans_timestamp group by issue_year, loan_status order by issue_year asc;

Year,loan_status,Loan
2007,Good,149
2007,Bad,145
2008,Bad,416
2008,Good,752
2009,Good,2046
2009,Bad,502
2010,Bad,1155
2010,Good,5224
2011,Good,8164
2011,Bad,1580


###### From the plot we can see the number of loans take have been increase drastically from 2007 till 2014 (2015 has incomplete data for the rest of the year). Similar pattern has followed by the distribution of the good and bad loans. The ratio maintained by them was consistent

##### No. of months (Term) vs good or bad  loans

In [34]:
display(sqlContext.sql("select  term as Term,  Loan_Status, count(Loan_status) as no_of_loans from loans_timestamp where Loan_status = 'Good' or Loan_status= 'Bad' group by Term, Loan_status "))

Term,Loan_Status,no_of_loans
36 months,Bad,21415
60 months,Bad,11881
60 months,Good,20585
36 months,Good,84529


###### The plot shows that there is a higher default rate for borrowers with a 60-month repayment term as compared to a 36-month term. This makes the term feature a desirable feature for predictive modeling

##### Interest range vs no. of good and bad loans

In [37]:
%sql
select 
CASE
WHEN int_rate >= 5 and int_rate <= 10 THEN '[5-10]'
         WHEN int_rate BETWEEN 10 AND 15 THEN '[10-15]' 
         WHEN int_rate BETWEEN 15 AND 20  THEN '[15-20]'
           WHEN int_rate BETWEEN 20 AND 25  THEN '[20-25]'
           WHEN int_rate BETWEEN 25 AND 30  THEN '[25-30]' END As Interest_Range, Loan_Status, count(int_rate) as no_of_loans
 FROM loans_timestamp where Loan_status = 'Good' or Loan_status= 'Bad' group by Interest_Range, Loan_Status order by count(int_rate) desc;

Interest_Range,Loan_Status,no_of_loans
[10-15],Good,46038
[15-20],Good,26965
[5-10],Good,25140
[15-20],Bad,12785
[10-15],Bad,12154
[20-25],Good,6406
[20-25],Bad,4902
[5-10],Bad,2823
[25-30],Bad,632
[25-30],Good,565


###### From the above graph we can see that the number of loans taken has an increasing trend for interest range 5-10% to 10-15% but a gradual decrease of count as the interest range increase from 15%. Also we can see that as the interest rate is increased the proportion of bad loans (people who didn't pay the loan) increase. Lower the interest rate better is the possibility of people paying the loan.

##### Annual Income range vs Number of Loans taken

In [40]:
%sql
select 
CASE
WHEN annual_inc BETWEEN 0 AND 20000 THEN '0-20k'
         WHEN annual_inc BETWEEN 20000 AND 40000 THEN '20k-40k' 
         WHEN annual_inc BETWEEN 40000 AND 60000  THEN '40k-60k'
           WHEN annual_inc BETWEEN 60000 AND 80000  THEN '60k-80k'
           WHEN annual_inc BETWEEN 80000 AND 100000  THEN '80k-100k' 
            WHEN annual_inc BETWEEN 100000 AND 120000  THEN '100k-120k' 
             WHEN annual_inc BETWEEN 120000 AND 140000  THEN '120k-140k' 
             WHEN annual_inc BETWEEN 140000 AND 160000  THEN '140k-160k' 
             WHEN annual_inc BETWEEN 160000 AND 180000  THEN '160k-180k' 
           ELSE 'ABOVE 180k' END As Annual_Income_Range,Loan_Status, count(annual_inc) as No_of_loans_issued
 FROM loans_timestamp  group by Annual_Income_Range, Loan_Status order by count(annual_inc) desc;

Annual_Income_Range,Loan_Status,No_of_loans_issued
40k-60k,Good,30160
60k-80k,Good,23083
20k-40k,Good,17558
80k-100k,Good,14816
40k-60k,Bad,10310
100k-120k,Good,7670
20k-40k,Bad,7394
60k-80k,Bad,6839
120k-140k,Good,3981
80k-100k,Bad,3836


###### From the above graph we can see that the number of loans taken is very high in the income range of *20k - 80k* and max in the range of *40k - 60k*. Also as the income is increased the proportion of people paying the loan (Good Loan) increased.

##### Number of Loans taken across each state

In [43]:
%sql
select count(id), addr_state from loans_timestamp group by addr_state;

count(id),addr_state
3347,AZ
1542,SC
1636,LA
2396,MN
5268,NJ
432,DC
1870,OR
4385,VA
590,RI
1192,KY


###### From the above graph we can see that most of the loans are taken by people in *California, New York and Texas* where most of the IT employees reside. Also from the above observations from the Loan purpose, more were debt_consolidation and credit cards, so we can co-relate that those people are more in these areas.

##### Default rate by home ownership

In [46]:
display(sqlContext.sql("select  home_ownership as Home_Ownership,  Loan_Status, count(Loan_status) as no_of_loans from loans_timestamp where (home_ownership = 'OWN' or home_ownership = 'OTHER' or home_ownership = 'RENT' or home_ownership = 'MORTGAGE') group by Home_Ownership, Loan_status order by no_of_loans desc "))

Home_Ownership,Loan_Status,no_of_loans
MORTGAGE,Good,53156
RENT,Good,42793
RENT,Bad,15570
MORTGAGE,Bad,14592
OWN,Good,9085
OWN,Bad,3087
OTHER,Good,64
OTHER,Bad,38


###### There does not seem to be a significant difference in the status of a loan based on home ownership but people with *Mortgages and Rented houses* are taking more loans

##### Credit Year and Interest Rate

In [49]:
display(sqlContext.sql("select  earliest_cr_line_year as Year,  avg(int_rate) as avg_interest from loans_timestamp where earliest_cr_line_year > 1968 group by Year order by Year asc"))

Year,avg_interest
1969,11.407532467532466
1970,12.507094017094019
1971,12.56010204081633
1972,12.563540372670804
1973,12.733584905660376
1974,12.315104602510461
1975,13.651239669421484
1976,12.861263157894737
1977,13.452300000000003
1978,13.220777385159007


###### From the above graph we can interpret that people who have longer credit history have maintained a better score hence on an average we can see a less interest rate when compared to people who have less credit history with less score

##### number of loans issued in each month across every year

In [52]:
display(sqlContext.sql("select  issue_month as month, issue_year as year,  count(id) as num_of_loans from loans_timestamp where issue_year<2014 group by month,year order by month asc "))

month,year,num_of_loans
1,2013,2789
1,2010,359
1,2012,1177
1,2009,134
1,2011,662
1,2008,144
2,2010,325
2,2012,1226
2,2009,148
2,2008,136


###### Though we cannot deduce any concrete interpretation out of the above graphs, we can see that in most of the years the maximum loans taken were on November month and minimum loans taken were on February month

#Pre-processing

In [55]:
#checking the count of the sample
df2.count()

Out[377]: 138410

Dropping total_pymnt_inv,total_rec_prncp,total_rec_col,total_rec_late_fee as they all add up to total_payment and they are redundant in the analysis

In [57]:
df2 = df2.drop('total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee')

In [58]:
display(df2.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df2.columns)))

id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,23,23,23,75918,23,23,0,116,23,0,0,0,0,0,0,450,0,126119,12,82,0,0,23,33047,33047,33047


In [59]:
#dropping rows with na values with in the ranges of 100 as we can afford them, when compared to the dataset size
df2=df2.na.drop(subset=['annual_inc','delinq_2yrs','earliest_cr_line','inq_last_6mths','open_acc','pub_rec','revol_util','total_acc','last_credit_pull_d','collections_12_mths_ex_med','acc_now_delinq'])

In [60]:
#Categorizing the prediction variable loan_status for better visualizability and prediction
from pyspark.sql import functions as F
df2 = df2.withColumn("Loan_status",F.when(col("loan_status") == "Charged Off",'Bad')
                    .when(col("loan_status") == "Default",'Bad')
                    .when(col("loan_status") == "In Grace Period",'late')
                    .when(col("loan_status") == "Does not meet the credit policy. Status:Charged Off",'Bad')
                    .when(col("loan_status") == "Late (31-120 days)",'late')
                    .when(col("loan_status") == "Late (16-30 days)",'late')
                    .otherwise('Good'))

In [61]:
#Considering charged off, default and doesn't meet credit policy as late
df2 = df2.filter((f.col('Loan_status') != "late"))

In [62]:
#Looking at the distribution of current, good vs bad in the main dataset
print(df2.filter(df2.Loan_status == 'Good').count()/df2.count())
print(df2.filter(df2.Loan_status == 'Bad').count()/df2.count())


0.8162616238180823
0.18373837618191763

From the ratios shown above, we can say that good loans are 81% and Bad loans are 18%. It is moderately imbalanced dataset and we are interested in the minority sample, which is bad

In [64]:
df2.createOrReplaceTempView('df2')

In [65]:
display(sqlContext.sql('select count(Loan_status) as status,sub_grade,Loan_status from df2 group by sub_grade,Loan_status order by status'))


status,sub_grade,Loan_status
53,G4,Bad
56,G5,Bad
65,G5,Good
69,G3,Bad
87,G4,Good
99,A1,Bad
100,G3,Good
110,G2,Bad
146,G2,Good
152,A2,Bad


In [66]:
display(sqlContext.sql('select count(Loan_status) as status,loan_status,verification_status from df2 group by verification_status,Loan_status order by status'))

status,loan_status,verification_status
6473,Bad,Not Verified
7112,Bad,Source Verified
9928,Bad,Verified
30018,Good,Source Verified
37174,Good,Verified
37265,Good,Not Verified


From the above plot, the median for all types of verification is almost same and it isn't affecting the loan_status variable much. So, removing the variable for analysis

In [68]:
display(sqlContext.sql('select count(Loan_status) as status,loan_status,pymnt_plan from df2 group by pymnt_plan,Loan_status order by status'))


status,loan_status,pymnt_plan
1,Good,y
23513,Bad,n
104456,Good,n


From the plot, the dataset is having very little info about y payment plan and it is a categorical variable and it will not affect the loan_status.

In [70]:
display(sqlContext.sql('select count(Loan_status) as status,loan_status,initial_list_status from df2 group by initial_list_status,Loan_status order by status'))


status,loan_status,initial_list_status
6471,Bad,w
17042,Bad,f
29384,Good,w
75073,Good,f


From the plot, the median for initial list status loans have higher median when compared to the W distribution

In [72]:
display(sqlContext.sql('select count(Loan_status) as status,loan_status,application_type from df2 group by application_type,Loan_status order by status'))

status,loan_status,application_type
1,Good,JOINT
23513,Bad,INDIVIDUAL
104456,Good,INDIVIDUAL


From the above plot, there is only one category dominant in the dataset, so it doesn't affect the loan_status variable and excluding this from analysis

In [74]:
#Creating a new feature called credit_line_time using issue_d and the earliest_credit_line date --> it helps us to know that from how many years a person has credit line opened
df2 = df2.withColumn('issue_D', from_unixtime(unix_timestamp('issue_d', 'MM/dd/yyy')))
df2 = df2.withColumn('earliest_cr_Line', from_unixtime(unix_timestamp('earliest_cr_line', 'MM/dd/yyy')))

In [75]:
df2 = df2.withColumn('issue_year',year(df2.issue_D))\
        .withColumn('credit_year',year(df2.earliest_cr_Line))


In [76]:
df2 = df2.withColumn('credit_line_time',(df2['issue_year'] - df2['credit_year']))

In [77]:
#Extracting the integer part of employment length and term
df2 = df2.withColumn("Emp_Length",regexp_extract(col('emp_length'),'\d+',0))
df2 = df2.withColumn("Term",regexp_extract(col('term'),'\d+',0))
df2 = df2.withColumn("emp_length",df2['Emp_Length'].cast("int"))
df2 = df2.withColumn("term",df2['Term'].cast("int"))

In [78]:
display(df2)

id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_D,Loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_Line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,issue_year,credit_year,credit_line_time
1077430,2500,2500,2500.0,60,15.27,59.83,C,C4,1,RENT,30000.0,Source Verified,2011-12-01 00:00:00,Bad,n,car,GA,1.0,0,1999-04-01 00:00:00,5,null,3,0,1687,9.4,4,f,0.0,0.0,1008.71,117.08,1.11,4/1/2013,119.66,null,9/1/2013,0,1,INDIVIDUAL,0,null,null,null,2011,1999,12
1077175,2400,2400,2400.0,36,15.96,84.33,C,C5,10,RENT,12252.0,Not Verified,2011-12-01 00:00:00,Good,n,small_business,IL,8.72,0,2001-11-01 00:00:00,2,null,2,0,2956,98.5,10,f,0.0,0.0,3003.653644,0.0,0.0,6/1/2014,649.91,null,1/1/2016,0,1,INDIVIDUAL,0,null,null,null,2011,2001,10
1077175,2400,2400,2400.0,36,15.96,84.33,C,C5,10,RENT,12252.0,Not Verified,2011-12-01 00:00:00,Good,n,small_business,IL,8.72,0,2001-11-01 00:00:00,2,null,2,0,2956,98.5,10,f,0.0,0.0,3003.653644,0.0,0.0,6/1/2014,649.91,null,1/1/2016,0,1,INDIVIDUAL,0,null,null,null,2011,2001,10
1071570,5375,5375,5350.0,60,12.69,121.45,B,B5,1,RENT,15000.0,Verified,2011-12-01 00:00:00,Bad,n,other,TX,18.08,0,2004-09-01 00:00:00,0,null,2,0,9279,36.5,3,f,0.0,0.0,1476.19,269.29,2.52,11/1/2012,121.45,null,3/1/2013,0,1,INDIVIDUAL,0,null,null,null,2011,2004,7
1069908,12000,12000,12000.0,36,12.69,402.54,B,B5,10,OWN,75000.0,Source Verified,2011-12-01 00:00:00,Good,n,debt_consolidation,CA,10.78,0,1989-10-01 00:00:00,0,null,12,0,23336,67.1,34,f,0.0,0.0,13943.08,0.0,0.0,9/1/2013,6315.3,null,8/1/2013,0,1,INDIVIDUAL,0,null,null,null,2011,1989,22
1062474,6000,6000,6000.0,36,11.71,198.46,B,B3,1,MORTGAGE,84000.0,Verified,2011-12-01 00:00:00,Good,n,medical,UT,18.44,2,2003-10-01 00:00:00,0,8,4,0,0,37.73,14,f,0.0,0.0,7164.499852,0.0,0.0,2/1/2015,16.98,null,7/1/2015,0,1,INDIVIDUAL,0,null,null,null,2011,2003,8
1069742,9200,9200,9200.0,36,6.03,280.01,A,A1,6,RENT,77385.19,Not Verified,2011-12-01 00:00:00,Good,n,debt_consolidation,CA,9.86,0,2001-01-01 00:00:00,0,null,8,0,7314,23.1,28,f,0.0,0.0,9459.96,0.0,0.0,7/1/2012,8061.1,null,7/1/2012,0,1,INDIVIDUAL,0,null,null,null,2011,2001,10
1069740,20250,20250,19142.16108,60,15.27,484.63,C,C4,3,RENT,43370.0,Verified,2011-12-01 00:00:00,Good,n,debt_consolidation,TX,26.53,0,1997-11-01 00:00:00,3,null,8,0,17813,85.6,22,f,0.0,0.0,27663.04267,0.0,0.0,8/1/2015,6024.09,null,12/1/2015,0,1,INDIVIDUAL,0,null,null,null,2011,1997,14
1069740,20250,20250,19142.16108,60,15.27,484.63,C,C4,3,RENT,43370.0,Verified,2011-12-01 00:00:00,Good,n,debt_consolidation,TX,26.53,0,1997-11-01 00:00:00,3,null,8,0,17813,85.6,22,f,0.0,0.0,27663.04267,0.0,0.0,8/1/2015,6024.09,null,12/1/2015,0,1,INDIVIDUAL,0,null,null,null,2011,1997,14
1069710,10000,10000,10000.0,36,11.71,330.76,B,B3,10,OWN,50000.0,Source Verified,2011-12-01 00:00:00,Good,n,credit_card,TX,11.18,0,1985-07-01 00:00:00,0,null,8,0,10056,82.4,21,f,0.0,0.0,11902.56157,0.0,0.0,1/1/2015,349.21,null,1/1/2016,0,1,INDIVIDUAL,0,null,null,null,2011,1985,26


In [79]:
#From the above visuzalization, I am removing the columns and we have enough info on derived column and based on domain knowledge
df2 = df2.drop('id','loan_amnt','funded_amnt_inv','pymnt_plan','application_type','mths_since_last_delinq','last_pymnt_d','next_pymnt_d','earliest_cr_line','issue_d','out_prncp_inv','total_pymnt_inv','issue_year','credit_year','last_credit_pull_d')

In [80]:
#Filtering the dataframe because we have null values in employee length
df2 = df2.filter(df2.emp_length.isNotNull())

In [81]:
#Using KNN Imputation to fill the null values as the columns are useful in the analysis. External library for imputation
from missingpy import KNNImputer
imputer = KNNImputer()

In [82]:
k = df2.select(['tot_cur_bal','tot_coll_amt','total_rev_hi_lim']).collect()
X_imputed = imputer.fit_transform(k)
X_1 = X_imputed.tolist()


/databricks/python/lib/python3.7/site-packages/missingpy/knnimpute.py:224: UserWarning: There are rows with more than 50.0% missing values. These rows are not included as donor neighbors.
 .format(self.row_max_missing * 100))
/databricks/python/lib/python3.7/site-packages/missingpy/knnimpute.py:282: UserWarning: There are rows with more than 50.0% missing values. The missing features in these rows are imputed with column means.
 .format(self.row_max_missing * 100))

In [83]:
df_dummy = df2.select(['tot_cur_bal','tot_coll_amt','total_rev_hi_lim'])

In [84]:
dt = spark.createDataFrame(X_1)
df_n1 = dt.selectExpr(["_1 as cur_bal","_2 as coll_amt","_3 as rev_hi_lim"])

In [85]:
ddf1 = df_n1.withColumn("row_id", monotonically_increasing_id())
df_dummy = df_dummy.withColumn("row_id", monotonically_increasing_id())
df_dummy = ddf1.join(df_dummy,on= 'row_id').drop("row_id")

In [86]:
ddf2 = df_dummy.withColumn("row_id", monotonically_increasing_id())
df2 = df2.withColumn("row_id", monotonically_increasing_id())
df2 = ddf2.join(df2,on ='row_id').drop("row_id")

In [87]:
df2 = df2.drop('tot_cur_bal','tot_coll_amt','total_rev_hi_lim')

In [88]:
#checking the dataframe after imputation
display(df2)

cur_bal,coll_amt,rev_hi_lim,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,Loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,total_pymnt,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,credit_line_time
139983.92454506565,145.9973674684919,29810.248916822973,13000,60,12.69,293.74,B,B5,1,MORTGAGE,30000.0,Source Verified,Good,debt_consolidation,NJ,13.72,0,0,17,0,17080,58.9,27,f,0.0,17402.23999,0.0,0.0,3596.46,0,1,0,9
139983.92454506565,145.9973674684919,29810.248916822973,8000,36,16.77,284.31,D,D2,1,RENT,62000.0,Not Verified,Good,debt_consolidation,VA,21.64,0,3,14,0,6088,66.9,20,f,0.0,10267.26214,0.0,0.0,606.54,0,1,0,5
139983.92454506565,145.9973674684919,29810.248916822973,22000,60,17.27,549.96,D,D3,2,RENT,54329.0,Verified,Good,debt_consolidation,NY,7.91,0,1,11,0,15808,77.1,29,f,0.0,30871.76906,0.0,0.0,12190.99,0,1,0,6
139983.92454506565,145.9973674684919,29810.248916822973,6000,36,8.9,190.52,A,A5,4,OWN,100000.0,Source Verified,Good,home_improvement,NY,8.32,0,0,5,0,9844,53.8,16,f,0.0,6857.415633,0.0,0.0,197.28,0,1,0,14
139983.92454506565,145.9973674684919,29810.248916822973,20000,60,14.65,472.14,C,C3,3,RENT,110000.0,Verified,Bad,debt_consolidation,CA,22.21,0,1,12,0,9365,31.2,36,f,0.0,11042.06,10098.06,1409.94,472.14,0,1,0,21
139983.92454506565,145.9973674684919,29810.248916822973,12000,60,13.49,276.06,C,C1,2,MORTGAGE,100000.0,Verified,Good,other,VA,9.3,0,0,12,0,15578,72.1,23,f,0.0,16069.52136,0.0,0.0,4783.64,0,1,0,12
139983.92454506565,145.9973674684919,29810.248916822973,30000,60,15.96,728.91,C,C5,5,RENT,55000.0,Verified,Good,debt_consolidation,NJ,15.4,0,6,7,0,2134,15.6,29,f,0.0,32320.76,0.0,0.0,28694.29,0,1,0,13
139983.92454506565,145.9973674684919,29810.248916822973,10000,36,13.49,339.31,C,C1,3,MORTGAGE,30000.0,Not Verified,Bad,home_improvement,CA,4.76,0,0,14,0,2227,31.8,18,f,0.0,9533.24,133.12,1.2292,500.0,0,1,0,6
139983.92454506565,145.9973674684919,29810.248916822973,12000,36,10.65,390.88,B,B2,10,RENT,69500.0,Source Verified,Good,debt_consolidation,NC,10.39,0,0,6,0,13448,85.7,19,f,0.0,12107.02,0.0,0.0,12107.27,0,1,0,20
139983.92454506565,145.9973674684919,29810.248916822973,6800,36,6.03,206.97,A,A1,3,RENT,124000.0,Not Verified,Good,debt_consolidation,VA,4.23,0,0,11,0,6822,14.7,23,f,0.0,6886.33,0.0,0.0,3266.85,0,1,0,10


In [89]:
#checking for correlation to see if there is multi collinearity
import pandas as pd
panda = df2.toPandas()

In [90]:
corrMatrix = panda.corr()
c = corrMatrix[corrMatrix >=0.8]
import seaborn as sn
sn.heatmap(c , annot=True, cmap="Greens", linewidths = 0.5)
plt.figure(figsize=(12,8))

from the pandas correlation matrix, there is multi collinearity and removing the variables(total_pymnt,installment and recoveries) having multi collinearity

In [92]:
#Finally checking for na values before modelling
display(df2.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df2.columns)))

cur_bal,coll_amt,rev_hi_lim,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,Loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,total_pymnt,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,credit_line_time
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [93]:
#dropping the multicolleniarity variables
df2 = df2.drop('total_pymnt','installment','recoveries')

In [94]:
#Factorizing the categorical variable loan_status for modelling
from pyspark.sql import functions as F
df2 = df2.withColumn("loan_status",F.when(col("Loan_status") == "Bad",1)
                    .otherwise(0))

**Modelling starts here**

In [96]:
#Splitting the train and test sets
train_data,test_data=df2.randomSplit([0.70,0.30])

In [97]:
#Looking at the distribution of bad in the train and test dataset
print(train_data.filter(train_data.loan_status == 1).count()/train_data.count())
print(test_data.filter(test_data.loan_status == 1).count()/test_data.count())

0.17619146577999445
0.1771682727946627

In [98]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier,RandomForestClassifier,GBTClassifier
from pyspark.ml.feature import VectorAssembler,StringIndexer,StandardScaler
from pyspark.ml import Pipeline

In [99]:
#Standard scaling for the numerical features
assembler = VectorAssembler(inputCols=['cur_bal',
 'coll_amt',
 'rev_hi_lim',
 'funded_amnt',
 'term',
 'int_rate',
 'emp_length',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'out_prncp',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'policy_code',
 'acc_now_delinq',
 'credit_line_time'],outputCol="noncategoricalfeatures")

train_data_norm = assembler.transform(train_data)
test_data_norm = assembler.transform(test_data)

In [100]:
Scalerizer=StandardScaler(withMean=True, withStd=True).setInputCol("noncategoricalfeatures").setOutputCol("Scaled_features")
Scale = Scalerizer.fit(train_data_norm)
train_data = Scale.transform(train_data_norm)
test_data = Scale.transform(test_data_norm)

In [101]:
#Converting the categorical features into factor variables[0,1..]
grade_indexer = StringIndexer(inputCol='sub_grade',outputCol='level',handleInvalid='keep')
ownership_indexer = StringIndexer(inputCol='home_ownership',outputCol='home',handleInvalid='keep')
add_indexer = StringIndexer(inputCol='addr_state',outputCol='add',handleInvalid='keep')
purpose_indexer = StringIndexer(inputCol='purpose',outputCol='purp',handleInvalid='keep')
higher_indexer = StringIndexer(inputCol='grade',outputCol='higher_level',handleInvalid='keep')
verification_indexer = StringIndexer(inputCol='verification_status',outputCol='verification',handleInvalid='keep')
initial_indexer = StringIndexer(inputCol='initial_list_status',outputCol='initial',handleInvalid='keep')

In [102]:
#Assembling all the input columns into a vector
Assembler = VectorAssembler(inputCols= ['Scaled_features','verification','level','home','add','purp','higher_level','verification','initial'],outputCol='features')

#Model Building for Four models

In [104]:
dt_model = DecisionTreeClassifier(labelCol='loan_status',featuresCol='features',maxBins=80)
lr_model = LogisticRegression(labelCol="loan_status",featuresCol="features")
random_model = RandomForestClassifier(labelCol="loan_status",featuresCol="features",maxBins=80)
gbt_model = GBTClassifier(labelCol="loan_status",featuresCol="features")

In [105]:
pipe = Pipeline(stages=[grade_indexer,ownership_indexer,add_indexer,purpose_indexer,higher_indexer,verification_indexer,initial_indexer,Assembler])

In [106]:
#Fitting the data and transforming
fit_pipe = pipe.fit(train_data)

In [107]:
train_data=fit_pipe.transform(train_data)

In [108]:
# #These three models took 17 mins to run
dt_fit = dt_model.fit(train_data.select(['features','loan_status']))
lr_fit = lr_model.fit(train_data.select(['features','loan_status']))
random_fit = random_model.fit(train_data.select(['features','loan_status']))

In [109]:
# #It is the costliest run and it took 35 mins
gbt_model = GBTClassifier(labelCol="loan_status",featuresCol="features",maxBins=80)
gbt_fit = gbt_model.fit(train_data.select(['features','loan_status']))

In [110]:
test_data=fit_pipe.transform(test_data)

In [111]:
#Storing the prediction results
results_dt = dt_fit.transform(test_data)
results_lr = lr_fit.transform(test_data)
results_random = random_fit.transform(test_data)

In [112]:
results_gbt = gbt_fit.transform(test_data)

Interpretation and Evaluation of all models

In [114]:
#decision tree
y_true_dt = results_dt.select(['Loan_status']).collect()
y_pred_dt = results_dt.select(['prediction']).collect()
#logistic regression
y_true_lr = results_lr.select(['Loan_status']).collect()
y_pred_lr = results_lr.select(['prediction']).collect()
#random forest
y_true_random = results_random.select(['Loan_status']).collect()
y_pred_random = results_random.select(['prediction']).collect()
#gbt classifier
y_true_gbt = results_gbt.select(['Loan_status']).collect()
y_pred_gbt = results_gbt.select(['prediction']).collect()

In [115]:
from sklearn.metrics import classification_report, confusion_matrix
print('Confusion matrix and report for decision tree')
print(classification_report(y_true_dt, y_pred_dt))
print(confusion_matrix(y_true_dt, y_pred_dt))
print('Confusion matrix and report for logistic regression')
print(classification_report(y_true_lr, y_pred_lr))
print(confusion_matrix(y_true_lr, y_pred_lr))
print('Confusion matrix and report for random forest model')
print(classification_report(y_true_random, y_pred_random))
print(confusion_matrix(y_true_random, y_pred_random))
print('Confusion matrix and report for GBT Classifier')
print(classification_report(y_true_gbt, y_pred_gbt))
print(confusion_matrix(y_true_gbt, y_pred_gbt))

Confusion matrix and report for decision tree
 precision recall f1-score support

 0 0.95 0.98 0.97 19980
 1 0.91 0.77 0.84 4302

 micro avg 0.95 0.95 0.95 24282
 macro avg 0.93 0.88 0.90 24282
weighted avg 0.95 0.95 0.94 24282

[[19664 316]
 [ 987 3315]]
Confusion matrix and report for logistic regression
 precision recall f1-score support

 0 0.95 0.98 0.97 19980
 1 0.91 0.75 0.82 4302

 micro avg 0.94 0.94 0.94 24282
 macro avg 0.93 0.87 0.89 24282
weighted avg 0.94 0.94 0.94 24282

[[19646 334]
 [ 1068 3234]]
Confusion matrix and report for random forest model
 precision recall f1-score support

 0 0.91 1.00 0.95 19980
 1 0.99 0.56 0.71 4302

 micro avg 0.92 0.92 0.92 24282
 macro avg 0.95 0.78 0.83 24282
weighted avg 0.93 0.92 0.91 24282

[[19960 20]
 [ 1906 2396]]
Confusion matrix and report for GBT Classifier
 precision recall f1-score support

 0 0.96 0.98 0.97 19980
 1 0.92 0.80 0.85 4302

 micro avg 0.95 0.95 0.95 24282
 macro avg 0.94 0.89 0.91 24282
weighted avg 0.95 0.95 0.95 24282

[[19672 308]
 [ 877 3425]]

From the confusion matrix of all the models, we can say that models have classified the classes decently in classifying the TPs and TNs. we can even see the f1 score as it is high for all the models

In [117]:
#Looking at the feature importances and coefficients for Logistic Regression to know what features are important to consider for the real world model
print('decison tree:',dt_fit.featureImportances)


decison tree: (31,[3,4,16,17,18,30],[0.01251206537611387,0.07503843808784891,0.01364907720865743,0.6278099417411446,0.2138851787638811,0.05710529882235429])

According to Decision tree, features funded_amnt,term,delinq,state,inquiries and acc_now delinq are important out of 31 features

In [119]:
print('Logistic Regression:',lr_fit.coefficients)


Logistic Regression: [-0.005596955979352934,-0.021098257141298038,0.0364490954295305,0.5469725908878191,0.41292279532822607,0.7652596463290984,0.006258825788481149,-0.2761866780260706,0.33602038465403966,0.053855943003338076,0.0936627772368491,0.051630644180804035,0.15716811573112252,-0.1296799201446141,-0.033420846087073276,0.014944202838698201,228.55966662085066,2865.865420144159,-9.74827314443036,0.034459060359614824,0.0,0.016439924376885226,0.08745553545032625,0.01905261905887451,-0.001994717367294483,0.030189437135950153,0.0020873078928109945,-0.053063448412556585,-0.1389060998336387,0.01905261905887451,1.3266907697050172]

In [120]:
print('Random Forest:',random_fit.featureImportances)


Random Forest: (31,[2,3,4,5,7,8,9,11,12,13,14,15,16,17,18,21,22,23,24,25,26,27,28,29,30],[0.00032904704456552787,0.003846695536992641,0.034601411960296126,0.03310849929687791,0.0009293629023683024,0.006023860026885052,0.0001623876762224922,9.465326063344419e-05,8.675463583109531e-05,2.333436220301616e-05,0.0005711131993618787,4.487392705692422e-05,0.026780474502087936,0.5764383604052193,0.30297546559638816,1.1660481490461385e-05,6.323475039137063e-05,0.00027291788100361047,0.003290221647586955,0.00036488292828248535,0.00020771008026804227,0.00016509909062916095,0.006323104693703135,0.0020904443815005364,0.0011944297321543733])

According to Random forest, features delinq,state and inquiries are affecting the loan_status variable

In [122]:
print('GBT Classifier:',gbt_fit.featureImportances)


GBT Classifier: (31,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,23,24,25,26,27,28,30],[2.229171029293565e-06,0.02809419416419428,7.930405471865106e-12,0.03426482070160292,0.009847908268744389,0.08181073751038075,2.0128378213455818e-09,0.006388778835782878,0.04196740614711474,0.003952243201683371,3.93128116186695e-09,0.0002517994849123777,0.018686707603821056,3.89720598802031e-06,2.559069434729617e-09,0.0012600275052965261,0.0032370254565877755,0.17084103001285603,0.44555673453993794,2.530735834305656e-19,0.0004905196536516735,0.0016321542019371855,0.07798059593592241,3.4033008505200088e-06,0.05526752034838682,0.006928077522335374,2.098107382984617e-16,0.011532180715864745])

According to GBT classifier forest, features delinq,state and inquiries are affecting the loan_status variable

In [124]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc = MulticlassClassificationEvaluator(labelCol="loan_status", predictionCol="prediction", metricName="accuracy")


In [125]:
accuracy = acc.evaluate(results_dt)
print("The accuracy of the decision tree classifier is {}".format(accuracy))
accuracy = acc.evaluate(results_lr)
print("The accuracy of the Logistic Regression classifier is {}".format(accuracy))
accuracy = acc.evaluate(results_random)
print("The accuracy of the Random Forest classifier is {}".format(accuracy))
accuracy = acc.evaluate(results_gbt)
print("The accuracy of the GBT classifier is {}".format(accuracy))

The accuracy of the decision tree classifier is 0.9463388518243967
The accuracy of the Logistic Regression classifier is 0.9422617576805864
The accuracy of the Random Forest classifier is 0.9206819866567828
The accuracy of the GBT classifier is 0.9511984185816654

In [126]:
f = MulticlassClassificationEvaluator(labelCol="loan_status", predictionCol="prediction", metricName="f1")


In [127]:
f1 = f.evaluate(results_dt)
print("The f1 score of the decision tree classifier is {}".format(f1))
f1 = f.evaluate(results_lr)
print("The f1 score of the Logistic Regression classifier is {}".format(f1))
f1 = f.evaluate(results_random)
print("The f1 score of the Random Forest classifier is {}".format(f1))
f1 = f.evaluate(results_gbt)
print("The f1 score of the GBT classifier is {}".format(f1))

The f1 score of the decision tree classifier is 0.9445125241104528
The f1 score of the Logistic Regression classifier is 0.9400899767575474
The f1 score of the Random Forest classifier is 0.9113356161201496
The f1 score of the GBT classifier is 0.9498130428579704

#Interpretation of the Model: Based on the feature importances and the classification matrix, we can say that all the models have performed well on this particular type of dataset, where good loans account to 80% and bad loans account to 20%, which is moderately imbalanced dataset. Out of all the models, GBT classifier has given the best accuracy of 95% and f1 of 94.67%

#Suggestions based on the modelling: Features delinq_2years and inq_last6 months have been shown to have high importance among other features in Decision tree, Random Forest and GBT Classifier. We hope that looking at these two things will have greater impact on the loan status and we advise the Lending club team to take care of these two based on our model.